In [1]:
import json
import w3lib.html
import random
import os
from datetime import datetime
import requests
import string
import calendar
import html
import locale 
from requests_html import HTMLSession
from requests_html import HTML
import validators
import pandas as pd
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
URL_SEED = 'https://www.elinformador.cl/category/actualidad-regional/page/1/'
XPATH_HREF = "//h3/a/@href"

In [3]:
class Crawler():
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    #Ruta XPATH hacia los enlaces que queremos recopilar
    xpath_href=XPATH_HREF

    #variable que permite listar todos los enlaces recopilado
    url_list = []
    
    def _init_(self, URL_SEED, XPATH_HREF):        
        pass

    
    def get(self,url):
        session = HTMLSession()
        header = dict()
        header['user-agent'] = random.choice(self.USER_AGENT_LIST)
        response = session.get(url,headers=header)        
        return response
    
    def parse(self, response):
        self.url_list = []
        urls = response.html.xpath(self.xpath_href)
        for url in urls:
            self.url_list.append(url)


In [4]:
urls=[]

for page in tqdm(range(3)): # revisemos sólo 2 páginas
    crawler = Crawler()
    response = crawler.get('https://www.elinformador.cl/category/actualidad-regional/page/'+str(page))
    crawler.parse(response)

    
    for url in crawler.url_list:
        urls.append(url)

    crawler.url_list = urls
    
crawler.url_list

  0%|          | 0/3 [00:00<?, ?it/s]

['https://www.elinformador.cl/nacional/que-tendencias-en-consumo-veremos-durante-el-2022-2/',
 'https://www.elinformador.cl/actualidad-regional/conaf-extiende-por-un-mes-la-prohibicion-de-realizar-quemas-agricolas-y-forestales-en-la-region-de-valparaiso/',
 'https://www.elinformador.cl/nacional/la-magia-de-la-champions-league/',
 'https://www.elinformador.cl/nacional/lo-ultimo-en-calzado-femenino/',
 'https://www.elinformador.cl/nacional/5-consejos-de-invideo-para-principiantes-en-la-edicion/',
 'https://www.elinformador.cl/actualidad-regional/500-personas-se-capacitan-sobre-tecnologias-de-riego-para-hacer-frente-a-la-sequia-en-valparaiso/',
 'https://www.elinformador.cl/agricultura/indap-flexibiliza-instrumentos-de-creditos-productores-pueden-adquirir-insumos-tecnologicos-para-digitalizar-sus-emprendimientos/',
 'https://www.elinformador.cl/agricultura/rescate-del-tomate-limachino-y-tecnificacion-del-riego-enfasis-de-la-visita-del-director-nacional-de-indap-a-region-de-valparaiso/',
 

In [5]:
len(urls)

195

In [6]:
urls[0]

'https://www.elinformador.cl/nacional/que-tendencias-en-consumo-veremos-durante-el-2022-2/'

In [7]:
URL_NEWS = urls[0]
XPATH_TITLE = "//h1[@class='entry-title']"
XPATH_CONTENT ="//div[@class='td-post-content']//p"

In [8]:
class Scraper:
    
    USER_AGENT_LIST = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1",
        "Mozilla/5.0 (X11; CrOS i686 2268.111.0) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.57 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1092.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.6 (KHTML, like Gecko) Chrome/20.0.1090.0 Safari/536.6",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/19.77.34.5 Safari/537.1",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.9 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/536.5 (KHTML, like Gecko) Chrome/19.0.1084.36 Safari/536.5",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1063.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1062.0 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.1 Safari/536.3",
        "Mozilla/5.0 (Windows NT 6.2) AppleWebKit/536.3 (KHTML, like Gecko) Chrome/19.0.1061.0 Safari/536.3",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24",
        "Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/535.24 (KHTML, like Gecko) Chrome/19.0.1055.1 Safari/535.24"
    ]

    query_extract_title = XPATH_TITLE
    query_extract_text =  XPATH_CONTENT

    def _init_(self):
        pass
   
    def get(self,url):
        headers = {'user-agent':random.choice(self.USER_AGENT_LIST) }
        session = HTMLSession()
        response = session.get(url,headers=headers) 
        return response
    
    def clean_text(self, html_text):
        text_without_tags = w3lib.html.remove_tags(html_text)
        text_without_escape_chars = w3lib.html.replace_escape_chars(text_without_tags)
        text_without_escape_chars = html.unescape(text_without_escape_chars)
        text_without_whitespace = text_without_escape_chars.strip()
        return text_without_whitespace

    
    #### main function ####
    def parse(self, response):
        
        #URL
        self.news_url = response.url
        
        #TITLE
        self.news_title = response.html.xpath(self.query_extract_title)[0].text
        self.news_title = self.clean_text(self.news_title)
       
        
        #CONTENT
        self.news_text = ""
        
        html_paragraphs = response.html.xpath(self.query_extract_text)
        
        for paragraph in html_paragraphs:
            paragraph_content = w3lib.html.remove_tags(paragraph.text)    
            text_cleaned = self.clean_text(paragraph_content)
            self.news_text = self.news_text+text_cleaned


In [9]:
scraper = Scraper()
response =scraper.get(URL_NEWS)

In [10]:
scraper.parse(response)

In [11]:
scraper.news_title

'¿Qué tendencias en consumo veremos durante el 2022?'

In [12]:
scraper.news_text

'Durante este año, las tendencias de consumo seguirán evolucionando a la par de la pandemia. Lo cierto es que las restricciones y flexibilizaciones impuestas por este período afectan mucho el comportamiento de los consumidores y, hasta que no llegue el fin de la propagación del virus, estas seguirán cambiando. Sin embargo, los especialistas ya están prediciendo qué tendencias en consumo veremos durante el 2022.Las restricciones de la pandemia ya no son tan estrictas como hasta hace unos meses. Esto traerá consigo una vuelta a las rutinas que manteníamos antes del 2020. Sin embargo, este retorno a los viejos hábitos se dará con un agregado particular. Las nuevas costumbres adquiridas se mantendrán con nosotros. ¿Qué quiere decir esto? Que, aunque se pueda ir a una tienda física a hacer compras, muchas personas seguirán optando por comprar online. Durante la pandemia, el proceso de digitalización se aceleró y llegó a sectores de la población cuya adquisición se suponía que iba a ser más 

In [13]:
#Verificamos que el scraper devuelve algo
assert(len(scraper.news_title)>0)
assert(len(scraper.news_text)>0)

In [14]:
df = pd.DataFrame(data={'url': [URL_NEWS],'title':[scraper.news_title], 'content':[scraper.news_text]})
df

,url,title,content
0,https://www.elinformador.cl/nacional/que-tende...,¿Qué tendencias en consumo veremos durante el ...,"Durante este año, las tendencias de consumo se..."


In [15]:
XPATH_TITLE = "//h1[@class='entry-title']"
XPATH_CONTENT ="//div[@class='td-post-content']//p"

for url in tqdm(urls):
    URL_NEWS = url
    scraper = Scraper()
    response =scraper.get(URL_NEWS)
    scraper.parse(response)
    df1 = pd.DataFrame(data={'url': url, 'title':scraper.news_title, 'content':scraper.news_text},index=['url', 'title', 'content'])
    df = df.append(df1, ignore_index=True)
df = df.drop_duplicates()


  0%|          | 0/195 [00:00<?, ?it/s]

In [16]:
df

,url,title,content
0,https://www.elinformador.cl/nacional/que-tende...,¿Qué tendencias en consumo veremos durante el ...,"Durante este año, las tendencias de consumo se..."
4,https://www.elinformador.cl/actualidad-regiona...,Conaf extiende por un mes la prohibición de re...,Las personas que infrinjan la restricción se e...
7,https://www.elinformador.cl/nacional/la-magia-...,La magia de la Champions League,"Por tanto, es evidente que se trata de una com..."
10,https://www.elinformador.cl/nacional/lo-ultimo...,Lo último en calzado femenino,"Todos los años, los cambios de estación nos tr..."
13,https://www.elinformador.cl/nacional/5-consejo...,5 consejos de Invideo para principiantes en la...,Hacer videos y montajes con imágenes es un ent...
...,...,...,...
568,https://www.elinformador.cl/comunas-de-aconcag...,"San Felipe: Con talleres de danza, deporte, mú...","Talleres de danza, actividades deportivas, ing..."
571,https://www.elinformador.cl/blogs-opinion/el-r...,El regalo ideal para esta Navidad,Por: Patricio Arroyo Jofre / Director de Escue...
574,https://www.elinformador.cl/comunas-de-aconcag...,Hospital de Los Andes mejora las condiciones p...,Madres y familiares agradecen adquisición de 6...
580,https://www.elinformador.cl/politica/abiertas-...,Abiertas postulaciones para el fondo de reacti...,Desde este miércoles 07 de Octubre hasta el pr...
